In [7]:
import numpy as np
import pandas as pd

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """ Receive dataframe. remove Nan's, remove columns that are usless """
    #Too many features with NaN's, remove columns which have more then 15000 Nan'n in column
    filtered_columns = df.columns[df.isna().sum() < 15000]
    filtered_df = df[filtered_columns]

    #remove some rows with a lot of Nan's
    test_filtered_df = filtered_df.dropna(subset='Acceleration')

    #columns that are usless.
    column_to_remove = ['Unnamed: 0.1','Assistance', 'Aspiration', 'Rear tires', 'Spare tire', 'Front tires',
                    'Urban autonomy', 'Generation', 'Front suspension', 'Rear suspension', 'Coupling', 'Valve command',
                    'Places', 'Disposition', 'Cylinders', 'Elastic element', 'Ipva R', 'Frontal area A', 'Engine code',
                    'Traction', 'Installation', 'Road autonomy', 'Engine power supply', 'Engine control activation',
                    'Gear change code', 'Corrected frontal area', 'Platform']

    # remove usless columns
    new_filtered_df = test_filtered_df.drop(columns=column_to_remove)
    #return clean dataframe
    return new_filtered_df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    ''' Receive dataframe. Transform values to numbers '''

    # columns that will be transformed same: Nan = 0 / standart equipment, optinal equipment = 1
    columns_to_transform = ['Hot air', 'Rev counter', 'Assisted direction', 'ABS brakes', 'Rear window', 'Central locking of the doors',
                        'Headrest for all occupants', 'Electric rearview mirror adjustment', 'Air conditioning',
                        'Bluetooth connection', 'Frontal Airbags', 'Steering wheel adjustment height',
                        'Electric front window control', 'Multifunctional steering wheel', 'Driver s seat with height adjustment',
                        'On board computer', 'Light in the trunk', 'Alloy wheels', 'USB connection', 'Radio',
                        'Folding rear seat', 'Perimeter anti theft alarm', 'Cooling liquid thermometer']
    df[columns_to_transform] = df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)

    ########################################Preprocess [Acceleration]#################################################################
    # Preprocess Acceleration (0100 km/h 3,8 s = 3.8)
    df['Acceleration'] = df['Acceleration'].str.extract(r'(\d+\,\d+)')
    # Replace ',' with '.' and convert to numeric
    df['Acceleration'] = df['Acceleration'].str.replace(',', '.').astype(float)
    # Rename Column
    df = df.rename(columns={'Acceleration': 'Acceleration 0100 km/h in S'})

    ##########################################Preproces values with mm/kg/cm/ ETC ####################################################
    #function to remove strings (mm/kg/cm/) and transform to floats
    def extract_float_value(value):
        try:
            if isinstance(value, float):
                return value
            else:
                float_value = value.split()[0].replace(',', '.')
                return float(float_value)
        except (ValueError, IndexError):
            return np.nan

    #transforming
    for column in df[['Weight/Torque', 'Weight', 'Weight/power', 'Max power regime.', 'Cylinder diameter',
                      'Fuel tank', 'Specific power', 'Maximum power', 'Length', 'Maximum torque', 'Width', 'Height',
                      'Specific torque', 'Minimum height from the ground', 'Piston course', 'Front gauge', 'Displacement',
                      'Turns diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Road consumption',
                      'Max torque regime', 'Car payload', 'Sidewall height', 'Unit displacement', 'Trunk', 'Urban']]:
        df[column] = df[column].apply(extract_float_value)
    #################################################################################################################################


    return df


In [8]:
df = pd.read_csv('/Users/bididudy/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')

/var/folders/ld/s5_rrf_n32q9rltztl0sz_fh0000gn/T/ipykernel_36895/867003968.py:1: DtypeWarning: Columns (24,25,26,35,43,53,59,63,74,75,85,89,108,110,116,120,122,124,136,141,147,150,151,162,163,177,178,180,195,214,221,226,234,241,246,259,266,270,271,273,278,283,287,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/bididudy/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')


In [9]:
df=clean_data(df)

In [10]:
df=preprocess(df)

/var/folders/ld/s5_rrf_n32q9rltztl0sz_fh0000gn/T/ipykernel_36895/214527678.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_transform] = df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


In [16]:
def process_guarantee_column(df):
    new_df = df.copy()

    # Remove 'year' or 'years' from 'Guarantee' column values
    new_df['Guarantee'] = new_df['Guarantee'].str.replace(r'\s*years?\s*', '', regex=True)

    # Rename the column to 'Guarantee in years'
    new_df = new_df.rename(columns={'Guarantee': 'Guarantee in years'})

    return new_df

In [17]:
df=process_guarantee_column(df)

In [21]:
import os

# Define the file path to the 'Downloads' directory
downloads_directory = os.path.expanduser('~/Downloads')

# Specify the full file path for saving the CSV file
csv_file_path = os.path.join(downloads_directory, 'df_7.csv')

# Save the DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)

In [20]:
df_object_columns = df.select_dtypes(include=['object'])
df_object_columns

,Compression ratio,Tuching,Guarantee in years,Drag coefficient,Propulsion,Car gearbox,Price R,Fuel,Devaluation,CNW Index,Reader score,Provenance,Settings,Car size
0,"10,71",NaN,1,"0,33",Combustion,Manual,811.249,Gasoline,"2,5","6,93","8,5",Imported,Coupe,Grande
1,"9,51",Mechanical,1,"0,36",Combustion,Manual,11.577,Gasoline,"8,27","64,45","8,8",National,Hatchback,Compact and portable
2,"9,81",Hydro,1,"0,3",Combustion,Manual,16.585,Gasoline,"2,94","77,95","8,4",National,Hatchback,Medium
4,"9,51",Hydro,1,"0,31",Combustion,Manual,33.527,Gasoline,"15,02","54,23","9,2",National,Hatchback,Medium
5,"10,31",Hydro,1,NaN,Combustion,Manual,12.087,Gasoline,"8,27","111,47","7,4",National,Hatchback,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,"10,51",Mechanical,5,NaN,Combustion,Automatic,102.090,Flex alcohol/gasoline,NaN,"973,96","7,0",National,Hatchback,Compact and portable
23882,"10,51",Mechanical,5,NaN,Combustion,Automatic,107.390,Flex alcohol/gasoline,NaN,"993,45","7,0",National,Hatchback,Compact and portable
23890,"10,51",Mechanical,5,NaN,Combustion,Automatic,117.090,Flex alcohol/gasoline,NaN,"1.001,37","7,0",National,Hatchback,Compact and portable
23902,161,NaN,1,NaN,Combustion,Manual,245.990,Diesel,NaN,"293,95","7,0",Imported,Van,Grande


## Converting string to float

In [22]:
def convert_columns_to_float(df, columns):
    """
    Replace commas with dots in specified columns and convert them to float.

    Parameters:
    - df: pandas.DataFrame.
    - columns: List of column names to be converted.

    Returns:
    - DataFrame with the specified columns converted to float.
    """
    for col in columns:
        # Check if the column exists in the DataFrame
        if col in df.columns:
            # Replace commas with dots and convert to float, coercing errors to NaN
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
        else:
            print(f"Column '{col}' not found in DataFrame.")
    return df


In [27]:
columns_to_convert = ['Guarantee in years','Drag coefficient', 'Price R', 'Devaluation', 'CNW Index', 'Reader score','Compression ratio']
cleaned_data = convert_columns_to_float(df=df, columns=columns_to_convert)

In [61]:
cleaned_data.columns

Index(['Hot air', 'Acceleration 0100 km/h in S', 'Length', 'Maximum torque',
       'Width', 'Rev counter', 'Compression ratio', 'Assisted direction',
       'CNW Ranking', 'ABS brakes', 'Rear window', 'Tuching', 'Weight/Torque',
       'Guarantee in years', 'Drag coefficient',
       'Central locking of the doors', 'Height', 'Valves per cylinder',
       'Specific torque', 'Minimum height from the ground', 'Propulsion',
       'Piston course', 'Car gearbox', 'Weight', 'Price R',
       'Headrest for all occupants', 'Front gauge', 'Fuel',
       'Electric rearview mirror adjustment', 'Weight/power', 'Displacement',
       'Cooling liquid thermometer', 'Max power regime.', 'Turns diameter',
       'Cylinder diameter', 'Air conditioning', 'Rear gauge',
       'Length between the axis', 'Bluetooth connection', 'Frontal Airbags',
       'Maximum speed', 'Fuel tank', 'Devaluation', 'Road consumption',
       'Max torque regime', 'Car payload', 'Steering wheel adjustment height',
       'Ele

In [28]:
df_object_columns = cleaned_data.select_dtypes(include=['object'])
df_object_columns

,Tuching,Propulsion,Car gearbox,Fuel,Provenance,Settings,Car size
0,NaN,Combustion,Manual,Gasoline,Imported,Coupe,Grande
1,Mechanical,Combustion,Manual,Gasoline,National,Hatchback,Compact and portable
2,Hydro,Combustion,Manual,Gasoline,National,Hatchback,Medium
4,Hydro,Combustion,Manual,Gasoline,National,Hatchback,Medium
5,Hydro,Combustion,Manual,Gasoline,National,Hatchback,Medium
...,...,...,...,...,...,...,...
23881,Mechanical,Combustion,Automatic,Flex alcohol/gasoline,National,Hatchback,Compact and portable
23882,Mechanical,Combustion,Automatic,Flex alcohol/gasoline,National,Hatchback,Compact and portable
23890,Mechanical,Combustion,Automatic,Flex alcohol/gasoline,National,Hatchback,Compact and portable
23902,NaN,Combustion,Manual,Diesel,Imported,Van,Grande


In [29]:
# # Define the file path to the 'Downloads' directory
# downloads_directory = os.path.expanduser('~/Downloads')

# # Specify the full file path for saving the CSV file
# csv_file_path = os.path.join(downloads_directory, 'df_8.csv')

# # Save the DataFrame to the CSV file
# df_object_columns.to_csv(csv_file_path, index=False)

## we will impute Unknown in Nan values in categorical data

In [30]:
def impute_missing_with_unknown(df, columns):
    """
    Impute missing values with 'Unknown' in specified columns.

    Parameters:
    - df: pandas.DataFrame.
    - columns: List of column names where missing values should be imputed.

    Returns:
    - DataFrame with missing values in the specified columns replaced with 'Unknown'.
    """
    for col in columns:
        df[col] = df[col].fillna('Unknown')
    return df

In [31]:
columns=[['Tuching', 'Fuel']]

In [69]:
cleaned_data=impute_missing_with_unknown(cleaned_data, columns)
cleaned_data


,Hot air,Acceleration 0100 km/h in S,Length,Maximum torque,Width,Rev counter,Compression ratio,Assisted direction,CNW Ranking,ABS brakes,...,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Carcode,Perimeter anti theft alarm
0,0,3.8,4580.0,66.3,2045.0,0,10.71,0,20549.0,0,...,516.0,50.0,0,6.0,2.4,2002.0,0,0,1,0
1,0,15.2,3644.0,8.5,1548.0,0,9.51,0,14268.0,0,...,250.0,290.0,0,5.0,10.0,2002.0,0,0,2,0
2,0,10.6,4177.0,17.3,1709.0,0,9.81,0,12266.0,0,...,500.0,370.0,0,5.0,7.2,2002.0,0,0,3,0
4,0,8.5,4149.0,21.4,1735.0,0,9.51,0,16041.0,0,...,445.0,330.0,0,5.0,6.8,2002.0,0,0,5,0
5,0,9.5,4187.0,16.7,1741.0,0,10.31,0,9142.0,0,...,437.0,370.0,0,5.0,9.0,2002.0,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,1,10.7,4015.0,17.5,1720.0,1,10.51,1,63.0,1,...,333.0,300.0,1,6.0,8.3,2024.0,1,1,23884,1
23882,1,10.7,4015.0,17.5,1720.0,1,10.51,1,54.0,1,...,333.0,300.0,1,6.0,8.3,2024.0,1,1,23885,1
23890,1,10.7,4015.0,17.5,1720.0,1,10.51,1,50.0,1,...,333.0,300.0,1,6.0,9.0,2024.0,1,1,23893,1
23902,1,14.9,5413.0,34.7,2050.0,1,161.00,1,1073.0,1,...,545.0,11500.0,1,6.0,10.0,2023.0,1,0,23905,1


## Applying one hot encoding to categorical data.

In [70]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# import pandas as pd


# # Define the categorical columns you want to encode
# categorical_columns = ['Tuching', 'Propulsion', 'Car gearbox', 'Fuel', 'Provenance', 'Settings', 'Car size']  # Adjust as needed

# # Initialize the ColumnTransformer to apply OneHotEncoding to the categorical columns
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(), categorical_columns)
#     ],
#     remainder='passthrough'  # Keep other columns unchanged
# )

# # Apply the transformations to the dataset
# # This will fit the OneHotEncoder to your categorical data and transform it,
# # while leaving the rest of your columns unchanged
# transformed_data = preprocessor.fit_transform(cleaned_data)

# # Get the new feature names for the one-hot encoded columns
# new_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

# # Combine the new feature names with the names of the non-transformed columns
# final_feature_names = list(new_feature_names) + [col for col in cleaned_data.columns if col not in categorical_columns]

# # Create a new DataFrame with the transformed data and the correct column names
# final_df = pd.DataFrame(transformed_data, columns=final_feature_names)

# # Display the first few rows of the final DataFrame
# final_df.head()

,Tuching_Hydro,Tuching_Mechanical,Tuching_Unknown,Propulsion_Combustion,Propulsion_Electric,Propulsion_Hybrid,Propulsion_Light Hybrid,Propulsion_Plug-in hybrid,Car gearbox_Automated,Car gearbox_Automatic,...,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Carcode,Perimeter anti theft alarm
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,516.0,50.0,0.0,6.0,2.4,2002.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,250.0,290.0,0.0,5.0,10.0,2002.0,0.0,0.0,2.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,500.0,370.0,0.0,5.0,7.2,2002.0,0.0,0.0,3.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,445.0,330.0,0.0,5.0,6.8,2002.0,0.0,0.0,5.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,437.0,370.0,0.0,5.0,9.0,2002.0,0.0,0.0,6.0,0.0


In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def encode_categorical_columns(data, categorical_columns):
    """
    Applies one-hot encoding to specified categorical columns in a DataFrame.

    Parameters:
    - data: pandas.DataFrame, the DataFrame containing the data.
    - categorical_columns: list, a list of column names to be one-hot encoded.

    Returns:
    - A new DataFrame with the specified columns one-hot encoded and original columns unchanged.
    """
    # Initialize the ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_columns)
        ],
        remainder='passthrough'  # Keep other columns unchanged
    )

    # Apply the transformations to the dataset
    transformed_data = preprocessor.fit_transform(data)

    # Get the new feature names for the one-hot encoded columns
    new_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

    # Combine the new feature names with the names of the non-transformed columns
    final_feature_names = list(new_feature_names) + [col for col in data.columns if col not in categorical_columns]

    # Create a new DataFrame with the transformed data and the correct column names
    final_df = pd.DataFrame(transformed_data, columns=final_feature_names, index=data.index)

    return final_df


In [73]:
# Define the categorical columns you want to encode
categorical_columns = ['Tuching', 'Propulsion', 'Car gearbox', 'Fuel', 'Provenance', 'Settings', 'Car size']

In [74]:
final_df=encode_categorical_columns(cleaned_data, categorical_columns)
final_df

,Tuching_Hydro,Tuching_Mechanical,Tuching_Unknown,Propulsion_Combustion,Propulsion_Electric,Propulsion_Hybrid,Propulsion_Light Hybrid,Propulsion_Plug-in hybrid,Car gearbox_Automated,Car gearbox_Automatic,...,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year,Radio,Folding rear seat,Carcode,Perimeter anti theft alarm
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,516.0,50.0,0.0,6.0,2.4,2002.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,250.0,290.0,0.0,5.0,10.0,2002.0,0.0,0.0,2.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,500.0,370.0,0.0,5.0,7.2,2002.0,0.0,0.0,3.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,445.0,330.0,0.0,5.0,6.8,2002.0,0.0,0.0,5.0,0.0
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,437.0,370.0,0.0,5.0,9.0,2002.0,0.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,333.0,300.0,1.0,6.0,8.3,2024.0,1.0,1.0,23884.0,1.0
23882,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,333.0,300.0,1.0,6.0,8.3,2024.0,1.0,1.0,23885.0,1.0
23890,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,333.0,300.0,1.0,6.0,9.0,2024.0,1.0,1.0,23893.0,1.0
23902,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,545.0,11500.0,1.0,6.0,10.0,2023.0,1.0,0.0,23905.0,1.0


In [71]:
# Define the file path to the 'Downloads' directory
downloads_directory = os.path.expanduser('~/Downloads')

# Specify the full file path for saving the CSV file
csv_file_path = os.path.join(downloads_directory, 'df_9.csv')

# Save the DataFrame to the CSV file
final_df.to_csv(csv_file_path, index=False)

In [75]:
nan_percentage_per_column = final_df.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': final_df.dtypes
})

# Display the new DataFrame
nan_info_df[0:50] # Cheking 50 by 50 to see whole list at a time.

,Column Name,NaN Percentage,Data Type
Tuching_Hydro,Tuching_Hydro,0.000000,float64
Tuching_Mechanical,Tuching_Mechanical,0.000000,float64
Tuching_Unknown,Tuching_Unknown,0.000000,float64
Propulsion_Combustion,Propulsion_Combustion,0.000000,float64
Propulsion_Electric,Propulsion_Electric,0.000000,float64
Propulsion_Hybrid,Propulsion_Hybrid,0.000000,float64
Propulsion_Light Hybrid,Propulsion_Light Hybrid,0.000000,float64
Propulsion_Plug-in hybrid,Propulsion_Plug-in hybrid,0.000000,float64
Car gearbox_Automated,Car gearbox_Automated,0.000000,float64
Car gearbox_Automatic,Car gearbox_Automatic,0.000000,float64


## putting 0 for empty cells in numerical columns.

In [76]:
def fill_missing_values_with_zero(data):
    """
    Fills all missing values in the DataFrame with 0.

    Parameters:
    - data: pandas.DataFrame, the DataFrame containing the data.

    Returns:
    - A new DataFrame with missing values filled with 0.
    """
    # Fill missing values with 0
    filled_data = data.fillna(0)

    return filled_data

In [77]:
final_cleaned_data=fill_missing_values_with_zero(final_df)

In [79]:
nan_percentage_per_column = final_cleaned_data.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': final_cleaned_data.dtypes
})

# Display the new DataFrame
nan_info_df[0:50] # Cheking 50 by 50 to see whole list at a time.

,Column Name,NaN Percentage,Data Type
Tuching_Hydro,Tuching_Hydro,0.0,float64
Tuching_Mechanical,Tuching_Mechanical,0.0,float64
Tuching_Unknown,Tuching_Unknown,0.0,float64
Propulsion_Combustion,Propulsion_Combustion,0.0,float64
Propulsion_Electric,Propulsion_Electric,0.0,float64
Propulsion_Hybrid,Propulsion_Hybrid,0.0,float64
Propulsion_Light Hybrid,Propulsion_Light Hybrid,0.0,float64
Propulsion_Plug-in hybrid,Propulsion_Plug-in hybrid,0.0,float64
Car gearbox_Automated,Car gearbox_Automated,0.0,float64
Car gearbox_Automatic,Car gearbox_Automatic,0.0,float64
